In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import pdfkit

In [78]:
! brew install Caskroom/cask/wkhtmltopdf

zsh:1: command not found: brew


In [50]:
june_alfa_df = pd.read_csv('/Users/xiangyikong/Documents/Knowsmoke/Knowsmoke Master Database List - June 2022 (RWAR).csv')

In [54]:
june_alfa_df['Combine_info'] = june_alfa_df['Year'].astype(str) + ' ' + june_alfa_df['Make'].astype(str) + ' ' + june_alfa_df['Model'].astype(str) + ' ' + june_alfa_df['Type'].astype(str)

In [58]:
test_df = june_alfa_df[['VIN',"Combine_info","Smoke History","Test Date"]][:5]
test_df

,VIN,Combine_info,Smoke History,Test Date
0,KM8J23A45KU943607,2019 HYUNDAI TUCSON SE,Positive,6/3/2022
1,ZARFAMBN7K7615376,2019 ALFA ROMEO GIULIA TI,Positive,6/1/2022
2,WBY8P4C51K7D10702,2019 BMW I3 REX,Negative,6/1/2022
3,ZARFAMBN4K7621054,2019 ALFA ROMEO GIULIA TI,Negative,6/1/2022
4,2C3CDXL96NH118792,2022 DODGE CHARGER SRT HELLCAT,Negative,6/2/2022


In [62]:
dealership = 'Russell West Brook Alfa Romeo Maserati of Van Nuys'
address = '5711 Van Nuys Blvd, Van Nuys, CA 91401.'

In [9]:
def create_certificate_for_passed_inspection(VIN,result,date,address,dealership,detail_info):
    with open('knowsmoke certificate/index.html', 'r') as file_1:
        soup = BeautifulSoup(file_1, features='html.parser')

    vechicle_VIN = soup.find('label', attrs={'class': "vin-num"})
    vechicle_VIN.contents[0].replace_with(VIN)# replace VIN
    vechicle_VIN.contents[2].replace_with(detail_info)# replace year make model type

    test_date = soup.find('label', attrs={'class': "test-date"})
    test_date.contents[0].replace_with(date)# replace test date

    dealership_name = soup.find('div', attrs={'class': "report-by"})
    dealership_name.contents[0].replace_with(dealership)# replace dealership full name

    dealership_address = soup.find('p')
    dealership_address.contents[0].replace_with(address)# replace dealership full address

    footnotes = soup.find('div', attrs={'class': "footer-det"})
    footnotes.contents[0].replace_with('I have reviewed and received a copy of the Knowsmoke Smoke History Report for this vehicle' \
    + '(VIN: ' + VIN + '), which is based on information collected by Knowsmoke and available as of ' + date + '.')# replace footnotes

    # result_paragraph = soup.find('p', attrs={'class': "tr-p"})
    # # if results are different, create another template. 
    # result_paragraph.contents[0].replace_with("This vehicle has undergone inspection and qualifies as a Certified Smoke-Free vehicle by Knowsmoke.")# replace result

    file_name = 'knowsmoke certificate/' + VIN+'_certificate.html'
    with open(file_name, 'wb') as file_1:
        file_1.write(soup.encode('utf-8'))
    return soup

In [8]:
def create_certificate_for_failed_inspection(VIN,result,date,address,dealership,detail_info):
    with open('knowsmoke certificate/index_for_failed_inspection.html', 'r') as file_1:
        soup = BeautifulSoup(file_1, features='html.parser')

    vechicle_VIN = soup.find('label', attrs={'class': "vin-num"})
    vechicle_VIN.contents[0].replace_with(VIN)# replace VIN
    vechicle_VIN.contents[2].replace_with(detail_info)# replace year make model type

    test_date = soup.find('label', attrs={'class': "test-date"})
    test_date.contents[0].replace_with(date)# replace test date

    dealership_name = soup.find('div', attrs={'class': "report-by"})
    dealership_name.contents[0].replace_with(dealership)# replace dealership full name

    dealership_address = soup.find('p')
    dealership_address.contents[0].replace_with(address)# replace dealership full address

    footnotes = soup.find('div', attrs={'class': "footer-det"})
    footnotes.contents[0].replace_with('I have reviewed and received a copy of the Knowsmoke Smoke History Report for this vehicle' \
    + '(VIN: ' + VIN + '), which is based on information collected by Knowsmoke and available as of ' + date + '.')# replace footnotes

    # result_paragraph = soup.find('p', attrs={'class': "tr-p"})
    # # if results are different, create another template. 
    # result_paragraph.contents[0].replace_with("This vehicle has been found to contain traces of tobacco-specific particles and does not qualify as a Certified Smoke-Free vehicle by Knowsmoke.")

    file_name = 'knowsmoke certificate/' + VIN+'_certificate.html'
    with open(file_name, 'wb') as file_1:
        file_1.write(soup.encode('utf-8'))
    return soup

In [97]:
test_df[test_df['Smoke History'] == 'Negative'].apply(lambda x: create_certificate_for_passed_inspection(VIN = x['VIN'], result = x['Smoke History'], \
    date = x['Test Date'], detail_info = x['Combine_info'],dealership = 'Russell West Brook Alfa Romeo Maserati of Van Nuys',\
        address = '5711 Van Nuys Blvd, Van Nuys, CA 91401.'), axis=1)
test_df[test_df['Smoke History'] == 'Positive'].apply(lambda x: create_certificate_for_failed_inspection(VIN = x['VIN'], result = x['Smoke History'], \
    date = x['Test Date'], detail_info = x['Combine_info'],dealership = 'Russell West Brook Alfa Romeo Maserati of Van Nuys',\
        address = '5711 Van Nuys Blvd, Van Nuys, CA 91401.'), axis=1)

0    [\n, html, \n, [\n, [\n, <script type="text/ja...
1    [\n, html, \n, [\n, [\n, <script type="text/ja...
dtype: object

In [17]:
options_pdf = {'enable-local-file-access': True, 
'page-size': 'A4', 'margin-top': '1in', 
'margin-bottom': '0.8in',
 'margin-right': '0.5in', 
 'margin-left': '0.5in', 
 'encoding': "UTF-8", 
 'header-html': '', 
 'footer-line': '', 
 'footer-html': '', 
 'footer-right': '[page] / [topage]', 
 'no-outline':None }
css = '/Users/xiangyikong/Documents/Knowsmoke/knowsmoke certificate/style.css'
pdfkit.from_file('/Users/xiangyikong/Documents/Knowsmoke/knowsmoke certificate/KM8J23A45KU943607_certificate.html',\
     '/Users/xiangyikong/Documents/Knowsmoke/knowsmoke certificate/KM8J23A45KU943607_certificate.pdf', \
        verbose=True, options=options_pdf, css =css)
print("="*50)

NameError: name 'pdfkit' is not defined

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

# For LAX Certificate

In [2]:
june_LAX_df = pd.read_csv('/Users/xiangyikong/Documents/Knowsmoke/Knowsmoke Master Database List - June 2022 (LAX).csv')
june_LAX_df.head(5)

,#,LAX#,VIN,Stock Number,Source,Prior Rental,Carfax Link,Year,Make,Model,Type,Miles,Owners,Smoke History,Test Date,Rapid Recon,Notes,Folder_Uploaded
0,1.0,190.0,2T2AZMAA4LC168142,KL3278,LAX,No,https://www.carfaxonline.com/cfm/Display_Deale...,2020.0,LEXUS,RX,350 BASE/350 F-SPORT,24374,1.0,Negative,6/1/2022,Yes,NaN,Uploaded
1,2.0,191.0,5N1AT2MV0KC754834,TL3106,LAX,Yes,https://www.carfaxonline.com/cfm/Display_Deale...,2019.0,NISSAN,ROGUE,S/SL/SV,68157,2.0,Negative,6/1/2022,Yes,NaN,Uploaded
2,3.0,192.0,WBA8E1G51HNU12807,PL3242,LAX,No,https://www.carfaxonline.com/cfm/Display_Deale...,2017.0,BMW,320I,NaN,"99,171",3.0,Negative,6/3/2022,Yes,NaN,Uploaded
3,4.0,193.0,5XYPG4A54GG094027,TL3110,LAX,No,https://www.carfaxonline.com/cfm/Display_Deale...,2016.0,KIA,SORENTO,LX/L,"83,761",3.0,Positive,6/2/2022,Yes,NaN,Uploaded
4,5.0,194.0,3KPFK4A71HE055194,PL3010,LAX,No,https://www.carfaxonline.com/cfm/Display_Deale...,2017.0,KIA,FORTE,LX,"53,679",1.0,Negative,6/1/2022,Yes,NaN,Uploaded


In [4]:
june_LAX_df = june_LAX_df[:5]
june_LAX_df['Year'] = june_LAX_df['Year'].astype(int)

In [5]:
june_LAX_df['Combine_info'] = june_LAX_df['Year'].astype(str) + ' ' + june_LAX_df['Make'].astype(str) + ' ' + june_LAX_df['Model'].astype(str) + ' ' + june_LAX_df['Type'].astype(str)

In [6]:
test_df = june_LAX_df[['VIN',"Combine_info","Smoke History","Test Date"]][:5]
test_df

,VIN,Combine_info,Smoke History,Test Date
0,2T2AZMAA4LC168142,2020 LEXUS RX 350 BASE/350 F-SPORT,Negative,6/1/2022
1,5N1AT2MV0KC754834,2019 NISSAN ROGUE S/SL/SV,Negative,6/1/2022
2,WBA8E1G51HNU12807,2017 BMW 320I nan,Negative,6/3/2022
3,5XYPG4A54GG094027,2016 KIA SORENTO LX/L,Positive,6/2/2022
4,3KPFK4A71HE055194,2017 KIA FORTE LX,Negative,6/1/2022


In [ ]:
dealership = 'LAX Automotive'
address = '1030 West Manchester BoulevardInglewood, CA 90301.'

In [10]:
test_df[test_df['Smoke History'] == 'Negative'].apply(lambda x: create_certificate_for_passed_inspection(VIN = x['VIN'], result = x['Smoke History'], \
    date = x['Test Date'], detail_info = x['Combine_info'],dealership = 'LAX Automotive',\
       address = '1030 West Manchester Blvd Inglewood, CA 90301.'), axis=1)
test_df[test_df['Smoke History'] == 'Positive'].apply(lambda x: create_certificate_for_failed_inspection(VIN = x['VIN'], result = x['Smoke History'], \
    date = x['Test Date'], detail_info = x['Combine_info'],dealership = 'LAX Automotive',\
        address = '1030 West Manchester Blvd Inglewood, CA 90301.'), axis=1)

3    [\n, html, \n, [\n, [\n, <script type="text/ja...
dtype: object

In [45]:
from bs4 import BeautifulSoup

with open('knowsmoke certificate/index.html', 'r') as file_1:
    #soup = BeautifulSoup(file_1, features='lxml')
    soup = BeautifulSoup(file_1, features='html.parser')

vechicle_VIN = soup.find('label', attrs={'class': "vin-num"})
test_date = soup.find('label', attrs={'class': "test-date"})
dealership = soup.find('div', attrs={'class': "report-by"})
dealership_address = soup.find('p')
footnotes = soup.find('div', attrs={'class': "footer-det"})
vechicle_VIN.contents[0].replace_with("11111111111111111")# replace VIN
vechicle_VIN.contents[2].replace_with("Year Make Model Type")# replace year make model type
test_date.contents[0].replace_with("Test Date")# replace test date
dealership.contents[0].replace_with("Dealership Full Name")# replace dealership full name
dealership_address.contents[0].replace_with("Dealership Full Address")# replace dealership full address
footnotes.contents[0].replace_with('I have reviewed and received a copy of the Knowsmoke Smoke History Report for this vehicle' \
    + '(VIN: ' + 'VIN_NUM' + '), which is based on information collected by Knowsmoke and available as of 06/24/2022 at 5:08PM (PST).')# replace footnotes

with open('Smoke-Free-Certificate_test1Jul22.html', 'wb') as file_1:
    file_1.write(soup.encode('utf-8'))